In [5]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import ScatterChart, Series, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

# Load and clean the dataset
df_raw = pd.read_excel('../nigel/output_freight_analysis/DIT_1CPT_freight_costs.xlsx')  # Replace with actual path
df = df_raw[[
    'quantity_sqyd',
    'vendor_estimated_cost',
    'actual_freight_cost',
    'ftl_cost',
    'ltl_only_cost',
    'optimal_cost'
]].copy()

# Create workbook and worksheet
wb = Workbook()
ws = wb.active
ws.title = "Freight Chart"

# Add data to worksheet
for r in dataframe_to_rows(df, index=False, header=True):
    ws.append(r)

# Set up chart
chart = ScatterChart()
#chart.title = "Freight Cost Overlay"
chart.x_axis.title = "Quantity (SQYD)"
chart.y_axis.title = "Freight Cost ($)"
chart.legend.position = "t"

max_row = ws.max_row
x_values = Reference(ws, min_col=1, min_row=2, max_row=max_row)

# Add each series
def add_series(title, y_col, marker=None):
    y_values = Reference(ws, min_col=y_col, min_row=2, max_row=max_row)
    series = Series(y_values, x_values, title=title)
    if marker:
        series.marker.symbol = marker
        series.graphicalProperties.line.noFill = True
    chart.series.append(series)

add_series("XGS Cost Curve", 2)
add_series("Historical Freight Spend", 3, marker="circle")
add_series("FTL Cost", 4)
add_series("LTL Cost", 5, marker="circle")
add_series("Optimal Cost", 6, marker="circle")

# Add chart to sheet
ws.add_chart(chart, "H2")

# Save to file
wb.save("freight_chart_notebook.xlsx")
